In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# For connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-31 01:39:09--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.53MB/s    in 0.2s    

2021-12-31 01:39:09 (5.53 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("level1software").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv.gz"), sep='\t', header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [5]:
# Count the number of records (rows) in the dataset.
df.count()

341931

In [6]:
# Remove duplicate rows and display head of dataframe
df = df.distinct()
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   38242199|R2LCT3GZXQF41Y|B0055A7RS8|      79858647|Individual Softwa...|        Software|          5|            0|          0|   N|                Y|Like the contacts...|Upgraded from Del...| 2015-08-31|
|         US|   22289015| R3VS4ADA5CN7S|B00CLVCY2I|      61914058|ESET NOD32 Antivi...|        Software|          5|    

In [7]:
# Count the number of records (rows) in the dataset.
df.count()

341931

In [8]:
# Transform the dataset to fit the tables in the schema file. Be sure the DataFrames match in data type and in column name
# Start with showing the data types
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'string'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'string'),
 ('total_votes', 'string'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [9]:
# Then, convert the data types that are not the same as the schema to the correct types
from pyspark.sql.types import IntegerType,BooleanType,DateType

corrected_df = df \
  .withColumn("customer_id" ,
              df["customer_id"]
              .cast(IntegerType()))   \
  .withColumn("product_parent" ,
              df["product_parent"]
              .cast(IntegerType()))   \
  .withColumn("star_rating" ,
              df["star_rating"]
              .cast(IntegerType()))   \
  .withColumn("helpful_votes" ,
              df["helpful_votes"]
              .cast(IntegerType()))   \
  .withColumn("total_votes" ,
              df["total_votes"]
              .cast(IntegerType()))   \
  .withColumn("review_date"  ,
              df["review_date"]
              .cast(DateType()))

corrected_df.show(5)
corrected_df.dtypes

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   38242199|R2LCT3GZXQF41Y|B0055A7RS8|      79858647|Individual Softwa...|        Software|          5|            0|          0|   N|                Y|Like the contacts...|Upgraded from Del...| 2015-08-31|
|         US|   22289015| R3VS4ADA5CN7S|B00CLVCY2I|      61914058|ESET NOD32 Antivi...|        Software|          5|    

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'date')]

In [10]:
# Lastly, transform the DataFrame into seperate DataFrames that match the tables we want (we'll do this one at a time)

# First DataFrame/Table: review_id_table
review_id_table = corrected_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(5)
review_id_table.dtypes

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2LCT3GZXQF41Y|   38242199|B0055A7RS8|      79858647| 2015-08-31|
| R3VS4ADA5CN7S|   22289015|B00CLVCY2I|      61914058| 2015-08-31|
|R3OVT9X8XPG7UK|   14733045|B002N5YI44|     970535244| 2015-08-29|
|R2314J99M3WLJH|   18753915|B000E28UT2|     642183116| 2015-08-27|
| RR085KKEB9JAW|   18048088|B003SEVL8U|     386618608| 2015-08-26|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [11]:
# Second DataFrame/Table: products
products = corrected_df.select(["product_id", "product_title"])
products.show(5)
products.dtypes

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0055A7RS8|Individual Softwa...|
|B00CLVCY2I|ESET NOD32 Antivi...|
|B002N5YI44|Hallmark Card Stu...|
|B000E28UT2|40 Years of X-Men...|
|B003SEVL8U|Webroot AntiVirus...|
+----------+--------------------+
only showing top 5 rows



[('product_id', 'string'), ('product_title', 'string')]

In [12]:
# Third DataFrame/Table: customers
customers = corrected_df.groupBy("customer_id").count()
customers = customers.withColumn("customer_count", customers["count"].cast(IntegerType()))
customers = customers.select("customer_id", "customer_count")
customers.show(5)
customers.dtypes

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    7455053|             9|
|   19365223|             1|
|   42863979|             1|
|   28424356|             1|
|   17533812|             1|
+-----------+--------------+
only showing top 5 rows



[('customer_id', 'int'), ('customer_count', 'int')]

In [13]:
# Last DataFrame/Table: vine_table
vine_table = corrected_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(5)
vine_table.dtypes

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2LCT3GZXQF41Y|          5|            0|          0|   N|
| R3VS4ADA5CN7S|          5|            0|          0|   N|
|R3OVT9X8XPG7UK|          2|            0|          0|   N|
|R2314J99M3WLJH|          5|            3|          3|   N|
| RR085KKEB9JAW|          1|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

In [14]:
# Ensure no duplicates exist in the newly divided up data
review_id_table = review_id_table.distinct()
products = products.distinct()
customers = customers.distinct()
vine_table = vine_table.distinct()

review_id_table.show(5)
products.show(5)
customers.show(5)
vine_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2G9RFFW051RWY|   13775565|B00OKSOLE4|     694520847| 2015-08-31|
|R3DV06I87GIM2P|   23020332|B00EDSI8HW|     990205232| 2015-08-31|
|R3DXOCYL3I2AU1|   12516845|B00EDSI8HW|     990205232| 2015-08-31|
|R2BD24NLZ12RVB|   41519365|B00RUXDRBY|     914980056| 2015-08-28|
| RF75V5QYQGWAG|   16058514|B01019TDJ8|     218396039| 2015-08-27|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00TQ71BXI|Moonlight Mysteri...|
|B00CHZG1FE|Image Line FL Stu...|
|B002JB3BC2|Anti-Virus & Anti...|
|B0113MZ1AC|Windows 8 64 Bit ...|
|B00003069T|ClueFinders 5th G...|
+----------+--------------------+
only showing top 5 rows

+-----------+--------------+
|customer_id

In [16]:
# Load the DataFrames that correspond to tables into an RDS instance
mode = "overwrite"
db_url = "jdbc:postgresql://<insert endpoint>:5432/big_data_challenge_l1_soft"
db_properties = {"user":"root", "password":"<insert password>", "driver":"org.postgresql.Driver"}

review_id_table.write.jdbc(url=db_url, table='review_id_table', mode=mode, properties=db_properties)
products.write.jdbc(url=db_url, table='products', mode=mode, properties=db_properties)
customers.write.jdbc(url=db_url, table='customers', mode=mode, properties=db_properties)
vine_table.write.jdbc(url=db_url, table='vine_table', mode=mode, properties=db_properties)